In [ ]:
#!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install pandas
#!pip install torch-summary

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Importing Necessary Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch #python #keras #tensorflow #pytorch
import torch.nn as nn
from torch.autograd import Variable

import pandas as pd

from sklearn.utils import shuffle
from torchsummary import summary

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/abalone (1).data", header=None)

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [ ]:
#Encoding all categorical features
from sklearn.preprocessing import OrdinalEncoder
Ordinal_encoder = OrdinalEncoder()

def jointlyTransform(df_columns):
    return Ordinal_encoder.fit_transform(df_columns)
df.iloc[:,[0]]=jointlyTransform(df.iloc[:,[0]])

In [ ]:
column_names = ['sex', 'length', 'diameter', 'height', 'whole weight', 'shucked weight', 'Viscera weight', 'shell weight', 'rings']
df.columns = column_names

In [ ]:
df.head()

,sex,length,diameter,height,whole weight,shucked weight,Viscera weight,shell weight,rings
0,2.0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,2.0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,0.0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,2.0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,1.0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


The last column denotes the number of rings, which is our target variable, treating it as a classification problem.

In [ ]:
# Split Train and Test Data

from sklearn.model_selection  import train_test_split
# y = dataframe['PRICE']
# X = dataframe.iloc[:,0:8]
y = df['rings']
X = df.iloc[:,0:-1]

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2923, 8) (1254, 8) (2923,) (1254,)


In [ ]:
#Change to numpy arraay.
X_train=X_train.values
y_train=y_train.values
X_test=X_test.values
y_test=y_test.values

In [ ]:
l = []
for i in y_train:
  if(i>=1 and i<= 8):
    l.append(0)
  elif(i==9 or i==10):
    l.append(1)
  else:
    l.append(2)
y_train2 = np.array(l)
l =[]
for i in y_test:
  if(i>=1 and i<= 8):
    l.append(0)
  elif(i==9 or i==10):
    l.append(1)
  else:
    l.append(2)
y_test2 = np.array(l)

In [ ]:
def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [ ]:
#Define training hyperprameters.
batch_size = 32 #sample batch
num_epochs = 1000 #number times dataset seen
learning_rate = 0.01
size_hidden_1 = 100 #neurons
size_hidden_2 = 100 #neurons
num_classes = 28

#Calculate some other hyperparameters based on data.
batch_no = len(X_train) // batch_size  #batches
cols = X_train.shape[1] #Number of columns in input matrix


In [ ]:
print(len(X_train))

2923


In [ ]:
#Create the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class Net(torch.nn.Module):
    def __init__(self, num_inputs, size_hidden_1, n_output):
        super(Net, self).__init__()
        self.hidden_layer_1 = torch.nn.Linear(num_inputs, size_hidden_1)   # hidden layer
        self.activation_1 = torch.nn.Tanh() # activation layer
        #self.hidden_layer_2 = torch.nn.Linear(num_inputs, size_hidden_2)   # hidden layer
        #self.activation_2 = torch.nn.Tanh() # activation layer
        #self.hidden_layer_3 = torch.nn.Linear(size_hidden_1, size_hidden_2)   # hidden layer
        #self.activation_3 = torch.nn.ReLU() # activation layer
        self.output_layer = torch.nn.Linear(size_hidden_1, n_output)   # output layer
        self.output_act = torch.nn.Sigmoid()

    def forward(self, x):
        x1 = self.activation_1(self.hidden_layer_1(x))     # activation function for hidden layer
        #x2= self.activation_2(self.hidden_layer_2(x))      # activation function for hidden layer
        #x3 = torch.add(x1,x2)
        x = self.output_act(self.output_layer(x1))                    # output
        return x
n_classes2=3
net = Net(cols, size_hidden_1, n_classes2)
#summary(net, (1, 4))

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
loss_func = torch.nn.CrossEntropyLoss()

In [ ]:
X_train_copy = X_train

In [ ]:
for epoch in range(num_epochs):
    #Shuffle just mixes up the dataset between epocs
    X_train, y_train2 = shuffle(X_train, y_train2)

    train_acc = 0.0
    running_loss = 0.0

    # Mini batch learning
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        inputs = Variable(torch.FloatTensor(X_train[start:end]))
        labels = Variable(torch.LongTensor(y_train2[start:end]))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        # print(labels)
        # loss = criterion(outputs, torch.unsqueeze(labels, dim=1))
        loss = loss_func(outputs, labels)

        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        acc = get_accuracy(outputs, labels, batch_size)
        train_acc += acc

    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
          %(epoch+1, running_loss / (i+1), train_acc/(i+1)))
    running_loss = 0.0





Epoch: 1 | Loss: 1.0978 | Train Accuracy: 39.05
Epoch: 2 | Loss: 1.0905 | Train Accuracy: 48.94
Epoch: 3 | Loss: 1.0841 | Train Accuracy: 50.21
Epoch: 4 | Loss: 1.0785 | Train Accuracy: 50.38
Epoch: 5 | Loss: 1.0736 | Train Accuracy: 50.48
Epoch: 6 | Loss: 1.0691 | Train Accuracy: 51.13
Epoch: 7 | Loss: 1.0646 | Train Accuracy: 51.58
Epoch: 8 | Loss: 1.0606 | Train Accuracy: 51.79
Epoch: 9 | Loss: 1.0566 | Train Accuracy: 52.23
Epoch: 10 | Loss: 1.0530 | Train Accuracy: 52.68
Epoch: 11 | Loss: 1.0494 | Train Accuracy: 52.68
Epoch: 12 | Loss: 1.0463 | Train Accuracy: 53.40
Epoch: 13 | Loss: 1.0428 | Train Accuracy: 53.64
Epoch: 14 | Loss: 1.0399 | Train Accuracy: 54.16
Epoch: 15 | Loss: 1.0367 | Train Accuracy: 54.57
Epoch: 16 | Loss: 1.0340 | Train Accuracy: 54.60
Epoch: 17 | Loss: 1.0310 | Train Accuracy: 54.77
Epoch: 18 | Loss: 1.0285 | Train Accuracy: 55.01
Epoch: 19 | Loss: 1.0258 | Train Accuracy: 55.43
Epoch: 20 | Loss: 1.0236 | Train Accuracy: 54.98
Epoch: 21 | Loss: 1.0208 | Tr

In [ ]:
def test_accuracy(X_test, y_test, batch_size):
  batch_no_test = len(X_test) // batch_size
  test_acc=0
  ##print(batch_no_test)
  for i in range(batch_no_test):
      start = i * batch_size
      end = start + batch_size
      inputs = Variable(torch.FloatTensor(X_test[start:end]))
      labels = Variable(torch.LongTensor(y_test[start:end]))

      # forward + backward + optimize
      outputs = net(inputs)
      acc = get_accuracy(outputs, labels, batch_size)
      test_acc += acc

  print(test_acc/(i+1))
test_accuracy(X_test, y_test2, batch_size)

55.84935897435897
